# Ingestão Cotação Histórica

Este notebook realiza a ingestão da cotação histórica (raw_cotahist) a partir dos dados do arquivo de cotação histórica da B3.

## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col, trim
import glob

## Start Spark Session

In [2]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [ ]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"

# Define as posições iniciais e finais de cada campo
positions = {"tipo_registro": (1, 2),
               "data_pregao": (3, 10),
               "codigo_bdi": (11, 12),
               "codigo_negociacao": (13, 24),
               "tipo_mercado": (25, 27),
               "nome_resumido_empresa": (28, 39),
               "especificacao_papel": (40, 49),
               "prazo_dias_mercado": (50, 52),
               "moeda_referencia": (53, 56),
               "preco_abertura_papel": (57, 69),
               "preco_maximo_papel": (70, 82),
               "preco_minimo_papel": (83, 95),
               "preco_medio_papel": (96, 108),
               "preco_ultimo_negocio": (109, 121),
               "preco_melhor_oferta_compra": (122, 134),
               "preco_melhor_oferta_venda": (135, 147),
               "numero_negocios_efetuados": (148, 152),
               "quantidade_total_titulos": (153, 170),
               "volume_total_titulos": (171, 188),
               "preco_exercicio_opcoes": (189, 201),
               "indicador_correcao_precos": (202, 202),
               "data_vencimento_opcoes": (203, 210),
               "fator_cotacao_papel": (211, 217),
               "preco_exercicio_pontos": (218, 230),
               "codigo_papel_sistema": (231, 242),
               "numero_distribuicao_papel": (243, 245)
			   }
        

## Read File

In [4]:
# Busca todos os arquivos que começam com 'COTAHIST' no diretório
file_list = glob.glob(f'{source_path}/COTAHIST*')

# Lê todos os arquivos encontrados
df = spark.read.text(file_list)

print(file_list)

['D:/Projetos/Jornada_financas_pessoais/data/source\\COTAHIST_A2025.TXT']


## Transformations

In [5]:
# Extrai os campos em formato fixo usando a função substring e trim
df = df.select(
    *[trim(substring("value", start, end - start + 1)).alias(field) 
      for field, (start, end) in positions.items()]
)

df = df.filter(col("tipo_registro") == "01")

In [6]:
print(df.count(), "registros lidos")

2396811 registros lidos


## Save Data Frame

In [ ]:
# grava os dados em formato Delta (substitui os anteriores)
df.write.format("delta").mode("overwrite").option("overwriteSchema", "false").save(delta_path_cotahist)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_cotahist}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cotahist


## Stop Spark Session

In [8]:
# Encerra a SparkSession
spark.stop()